In [1]:
import numpy as np
import pickle
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

# Simulation

The plant is controlled by a neural network.  
Calling `gcas_simulation(initial_state, initial_time)` for a 15 seconds simulation.  

In [2]:
from utils.simulation import gcas_simulation, initial_space
from CoRec.envs.F16.AeroBenchVVPython.code.plot import plot3d_anim, plot3d

l, h = initial_space()
initial_state = (np.random.uniform(l, h))
passed, times, states, modes, ps_list, Nz_list, u_list = gcas_simulation(initial_state, 0)
print(f"Passed: {passed}")

Passed: True


In [3]:
with open("results/traj.pkl", "wb") as f:
    pickle.dump([times, states], f)

Generating animation will take several minutes.

In [4]:
# plot3d_anim(times, states, modes, ps_list, Nz_list, filename="results/f16_passed.gif")

![F16 Passed](results/f16_passed.gif)

# Falsify

Adding perturbation around above simulation trajectory for falsifing the control system.  
Support multiprocess attack, set the threads number and attack step like following code.  
Paralization is based on Ray. 

In [5]:
from falsify.para import apply_para
from falsify.bo import F16SimulationFunction, EvaluatedFunction, attack
attack_threads = 2
attack_step = 20

2020-10-09 22:07:45,119	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


In [8]:
unsafe_states = []
for i in range(int(attack_step/attack_threads)):
    eval_fns = [EvaluatedFunction(F16SimulationFunction(t)) for t in times[i*attack_threads:(i+1)*attack_threads]]
    delta = (h-l)/2
    attack_traj = np.array(states)[i*attack_threads:(i+1)*attack_threads]
    attack_spaces = [[attack_traj[j]-delta, attack_traj[j]+delta] for j in range(attack_threads)]
    args = [[eval_fn, space] for eval_fn, space in zip(eval_fns, attack_spaces)]
    _unsafe_states = apply_para(attack, args)
    unsafe_states.append(_unsafe_states)

Find one unsafe state when simulation runs to 0.19s.

In [18]:
unsafe_t = unsafe_states[-1][-1][0][0]
unsafe_state = unsafe_states[-1][-1][0][1]
print(f"Find unsafe state: \n{np.array(unsafe_state)} \nwhen t = {str(unsafe_t)[:4]} s")

Find unsafe state: 
[ 5.65270094e+02  2.58062642e-02  2.21452047e-03  7.92783488e-01
 -1.25981053e+00 -8.02283545e-01 -1.24473254e-03 -8.09482788e-03
  2.69601764e-03  2.01781729e+01 -2.37740480e+01  3.49880148e+03
  7.26968119e+00  3.13598870e-02 -5.87374181e-05  2.98813527e-04] 
when t = 0.19 s


In [21]:
passed, times, states, modes, ps_list, Nz_list, u_list = gcas_simulation(unsafe_state, unsafe_t)
print(f"Passed: {passed}")

Passed: False


In [23]:
# plot3d_anim(times, states, modes, ps_list, Nz_list, filename="results/f16_unsafe.gif")

![F16 Passed](results/f16_unsafe.gif)